In [2]:
import pandas as pd

In [3]:
import numpy as np

In [331]:
from sklearn.preprocessing import LabelEncoder

In [379]:
import random

In [382]:
import xgboost as xgb

In [415]:
import pickle

In [438]:
import math

In [425]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [388]:
train = pd.read_csv('train.csv')

/root/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
train.head()

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western...",1131660440785968503_1472830385,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472830385,1,1472830385
1,Organic Search,20160902,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Aust...",377306020877927890_1472880147,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472880147,1,1472880147
2,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3895546263509774583,"{""continent"": ""Europe"", ""subContinent"": ""South...",3895546263509774583_1472865386,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472865386,1,1472865386
3,Organic Search,20160902,"{""browser"": ""UC Browser"", ""browserVersion"": ""n...",4763447161404445595,"{""continent"": ""Asia"", ""subContinent"": ""Southea...",4763447161404445595_1472881213,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472881213,1,1472881213
4,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",27294437909732085,"{""continent"": ""Europe"", ""subContinent"": ""North...",27294437909732085_1472822600,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472822600,2,1472822600


In [430]:
test = pd.read_csv('test.csv')

/root/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
test.head()

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6167871330617112363,"{""continent"": ""Asia"", ""subContinent"": ""Southea...",6167871330617112363_1508151024,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""4"", ""pageviews"": ""4""}","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508151024,2,1508151024
1,Organic Search,20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",0643697640977915618,"{""continent"": ""Europe"", ""subContinent"": ""South...",0643697640977915618_1508175522,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""5"", ""pageviews"": ""5"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508175522,1,1508175522
2,Organic Search,20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6059383810968229466,"{""continent"": ""Europe"", ""subContinent"": ""Weste...",6059383810968229466_1508143220,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""7"", ""pageviews"": ""7"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508143220,1,1508143220
3,Organic Search,20171016,"{""browser"": ""Safari"", ""browserVersion"": ""not a...",2376720078563423631,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",2376720078563423631_1508193530,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""8"", ""pageviews"": ""4"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508193530,1,1508193530
4,Organic Search,20171016,"{""browser"": ""Safari"", ""browserVersion"": ""not a...",2314544520795440038,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",2314544520795440038_1508217442,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""9"", ""pageviews"": ""4"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508217442,1,1508217442


In [433]:
test.shape

(804684, 12)

In [436]:
test['fullVisitorId'].nunique()

652237

In [431]:
sample_sub = pd.read_csv('sample_submission.csv')

In [432]:
sample_sub.shape

(617242, 2)

In [437]:
sample_sub['fullVisitorId'].nunique()

617242

In [434]:
sample_sub.columns

Index(['fullVisitorId', 'PredictedLogRevenue'], dtype='object')

# Basic Analysis

In [10]:
set(train.columns) - set(test.columns)

set()

In [11]:
train['channelGrouping'].value_counts()

Organic Search    381561
Social            226117
Direct            143026
Referral          104838
Paid Search        25326
Affiliates         16403
Display             6262
(Other)              120
Name: channelGrouping, dtype: int64

In [12]:
test['channelGrouping'].value_counts()

Organic Search    357402
Direct            130108
Social            128854
Referral          106469
Display            45021
Paid Search        20301
Affiliates         16512
(Other)               17
Name: channelGrouping, dtype: int64

In [15]:
(min(train['date']),max(train['date']))

(20160801, 20170801)

In [16]:
(min(test['date']),max(test['date']))

(20170802, 20180430)

In [17]:
train.shape

(903653, 12)

In [18]:
test.shape

(804684, 12)

In [19]:
train['fullVisitorId'].nunique()

742735

In [20]:
test['fullVisitorId'].nunique()

652237

In [22]:
train['sessionId'].nunique()

902755

In [23]:
test['sessionId'].nunique()

803863

In [24]:
train['socialEngagementType'].value_counts()

Not Socially Engaged    903653
Name: socialEngagementType, dtype: int64

In [25]:
test['socialEngagementType'].value_counts()

Not Socially Engaged    804684
Name: socialEngagementType, dtype: int64

In [27]:
train['visitId'].nunique()

886303

In [28]:
test['visitId'].nunique()

779504

In [29]:
train['visitNumber'].nunique()

384

In [30]:
test['visitNumber'].nunique()

446

In [41]:
train["transactionRevenue"] = train['totals'].apply(lambda x : eval(x)["transactionRevenue"] if "transactionRevenue" in x else np.nan)

In [43]:
pd.isnull(train["transactionRevenue"]).value_counts()

True     892138
False     11515
Name: transactionRevenue, dtype: int64

In [49]:
train[~pd.isnull(train["transactionRevenue"])]['fullVisitorId'].nunique()

10366

In [44]:
test["transactionRevenue"] = test['totals'].apply(lambda x : eval(x)["transactionRevenue"] if "transactionRevenue" in x else np.nan)

In [45]:
pd.isnull(test["transactionRevenue"]).value_counts()

True    804684
Name: transactionRevenue, dtype: int64

In [50]:
len(set(train['fullVisitorId']).intersection(test['fullVisitorId']))

5296

In [85]:
train_use = train[pd.notnull(train['transactionRevenue'])]
train_use.shape

(11515, 13)

In [127]:
len(set(train_use['fullVisitorId']).intersection(test['fullVisitorId']))

365

# JSON parsing

In [ ]:
#device, geoNetwork, totals, trafficSource

In [86]:
device_cols = list(train_use['device'].apply(lambda x : list(eval(x.replace('false','False').replace('true','True')).keys())))

In [92]:
device_all_cols = list(set([j for i in device_cols for j in i]))
device_all_cols

['operatingSystemVersion',
 'mobileDeviceModel',
 'mobileInputSelector',
 'mobileDeviceInfo',
 'browserVersion',
 'screenColors',
 'browserSize',
 'browser',
 'isMobile',
 'mobileDeviceMarketingName',
 'screenResolution',
 'operatingSystem',
 'mobileDeviceBranding',
 'language',
 'deviceCategory',
 'flashVersion']

In [93]:
geoNetwork_cols = list(train_use['geoNetwork'].apply(lambda x : list(eval(x.replace('false','False').replace('true','True')).keys())))
geoNetwork_all_cols = list(set([j for i in geoNetwork_cols for j in i]))
geoNetwork_all_cols

['country',
 'city',
 'longitude',
 'latitude',
 'networkLocation',
 'continent',
 'metro',
 'subContinent',
 'cityId',
 'networkDomain',
 'region']

In [95]:
totals_cols = list(train_use['totals'].apply(lambda x : list(eval(x.replace('false','False').replace('true','True')).keys())))
totals_all_cols = list(set([j for i in totals_cols for j in i]))
totals_all_cols

['pageviews', 'hits', 'newVisits', 'transactionRevenue', 'visits']

In [96]:
trafficSource_cols = list(train_use['trafficSource'].apply(lambda x : list(eval(x.replace('false','False').replace('true','True')).keys())))
trafficSource_all_cols = list(set([j for i in trafficSource_cols for j in i]))
trafficSource_all_cols

['adContent',
 'campaign',
 'source',
 'keyword',
 'referralPath',
 'isTrueDirect',
 'adwordsClickInfo',
 'medium']

## Unstructured to Structured data conversion

In [114]:
for i in device_all_cols:
    train_use['device_'+i] = train_use['device'].apply(lambda x : eval(x.replace('false','False').replace('true','True'))[i] if i in x else '')

/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [116]:
for i in geoNetwork_all_cols:
    train_use['geoNetwork_'+i] = train_use['geoNetwork'].apply(lambda x : eval(x.replace('false','False').replace('true','True'))[i] if i in x else '')

/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [118]:
for i in totals_all_cols:
    train_use['totals_'+i] = train_use['totals'].apply(lambda x : eval(x.replace('false','False').replace('true','True'))[i] if i in x else '')

/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [120]:
for i in trafficSource_all_cols:
    train_use['trafficSource_'+i] = train_use['trafficSource'].apply(lambda x : eval(x.replace('false','False').replace('true','True'))[i] if i in x else '')

/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [123]:
del train_use['device']
del train_use['geoNetwork']
del train_use['totals']
del train_use['trafficSource']
train_use.to_csv('train_use.csv', index = False)

In [139]:
train_use.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,transactionRevenue,device_operatingSystemVersion,...,totals_transactionRevenue,totals_visits,trafficSource_adContent,trafficSource_campaign,trafficSource_source,trafficSource_keyword,trafficSource_referralPath,trafficSource_isTrueDirect,trafficSource_adwordsClickInfo,trafficSource_medium
752,Direct,20160902,6194193421514403509,6194193421514403509_1472843572,Not Socially Engaged,1472843572,1,1472843572,37860000,not available in demo dataset,...,37860000,1,,(not set),(direct),,,True,{'criteriaParameters': 'not available in demo ...,(none)
753,Organic Search,20160902,5327166854580374902,5327166854580374902_1472844906,Not Socially Engaged,1472844906,3,1472844906,306670000,not available in demo dataset,...,306670000,1,,(not set),google,(not provided),,True,{'criteriaParameters': 'not available in demo ...,organic
799,Referral,20160902,8885051388942907862,8885051388942907862_1472827393,Not Socially Engaged,1472827393,7,1472827393,68030000,not available in demo dataset,...,68030000,1,,(not set),mall.googleplex.com,,/,True,{'criteriaParameters': 'not available in demo ...,referral
802,Referral,20160902,0185467632009737931,0185467632009737931_1472846398,Not Socially Engaged,1472846398,6,1472846398,26250000,not available in demo dataset,...,26250000,1,,(not set),mall.googleplex.com,,/,True,{'criteriaParameters': 'not available in demo ...,referral
859,Referral,20160902,3244885836845029978,3244885836845029978_1472824817,Not Socially Engaged,1472824817,4,1472824817,574150000,not available in demo dataset,...,574150000,1,,(not set),mall.googleplex.com,,/,True,{'criteriaParameters': 'not available in demo ...,referral


In [140]:
train_use.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId',
       'socialEngagementType', 'visitId', 'visitNumber', 'visitStartTime',
       'transactionRevenue', 'device_operatingSystemVersion',
       'device_mobileDeviceModel', 'device_mobileInputSelector',
       'device_mobileDeviceInfo', 'device_browserVersion',
       'device_screenColors', 'device_browserSize', 'device_browser',
       'device_isMobile', 'device_mobileDeviceMarketingName',
       'device_screenResolution', 'device_operatingSystem',
       'device_mobileDeviceBranding', 'device_language',
       'device_deviceCategory', 'device_flashVersion', 'geoNetwork_country',
       'geoNetwork_city', 'geoNetwork_longitude', 'geoNetwork_latitude',
       'geoNetwork_networkLocation', 'geoNetwork_continent',
       'geoNetwork_metro', 'geoNetwork_subContinent', 'geoNetwork_cityId',
       'geoNetwork_networkDomain', 'geoNetwork_region', 'totals_pageviews',
       'totals_hits', 'totals_newVisits', 'totals_transactionRev

In [176]:
train_use['trafficSource_adwordsClickInfo'].apply(lambda x : len(eval(str(x)))).value_counts()

1    11059
6      390
7       64
2        2
Name: trafficSource_adwordsClickInfo, dtype: int64

In [179]:
trafficSource_adwordsClickInfo_cols = list(train_use['trafficSource_adwordsClickInfo'].apply(lambda x : list(eval(str(x).replace('false','False').replace('true','True')).keys())))
trafficSource_adwordsClickInfo_all_cols = list(set([j for i in trafficSource_adwordsClickInfo_cols for j in i]))
trafficSource_adwordsClickInfo_all_cols

['slot',
 'page',
 'isVideoAd',
 'adNetworkType',
 'criteriaParameters',
 'targetingCriteria',
 'gclId']

In [180]:
for i in trafficSource_adwordsClickInfo_all_cols:
    train_use['trafficSource_adwordsClickInfo_'+i] = train_use['trafficSource_adwordsClickInfo'].apply(lambda x : eval(str(x).replace('false','False').replace('true','True'))[i] if i in x else '')

/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [184]:
del train_use['trafficSource_adwordsClickInfo']

In [185]:
train_use.to_csv('train_use_v2.csv', index = False)

In [124]:
for i in device_all_cols:
    test['device_'+i] = test['device'].apply(lambda x : eval(x.replace('false','False').replace('true','True'))[i] if i in x else '')

for i in geoNetwork_all_cols:
    test['geoNetwork_'+i] = test['geoNetwork'].apply(lambda x : eval(x.replace('false','False').replace('true','True'))[i] if i in x else '')

for i in totals_all_cols:
    test['totals_'+i] = test['totals'].apply(lambda x : eval(x.replace('false','False').replace('true','True'))[i] if i in x else '')

for i in trafficSource_all_cols:
    test['trafficSource_'+i] = test['trafficSource'].apply(lambda x : eval(x.replace('false','False').replace('true','True'))[i] if i in x else '')

del test['device']
del test['geoNetwork']
del test['totals']
del test['trafficSource']


In [126]:
test.to_csv('test_new.csv', index = False)

In [186]:
for i in trafficSource_adwordsClickInfo_all_cols:
    test['trafficSource_adwordsClickInfo_'+i] = test['trafficSource_adwordsClickInfo'].apply(lambda x : eval(str(x).replace('false','False').replace('true','True'))[i] if i in x else '')

In [192]:
test.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,transactionRevenue,device_operatingSystemVersion,...,trafficSource_isTrueDirect,trafficSource_adwordsClickInfo,trafficSource_medium,trafficSource_adwordsClickInfo_slot,trafficSource_adwordsClickInfo_page,trafficSource_adwordsClickInfo_isVideoAd,trafficSource_adwordsClickInfo_adNetworkType,trafficSource_adwordsClickInfo_criteriaParameters,trafficSource_adwordsClickInfo_targetingCriteria,trafficSource_adwordsClickInfo_gclId
0,Organic Search,20171016,6167871330617112363,6167871330617112363_1508151024,Not Socially Engaged,1508151024,2,1508151024,NaN,not available in demo dataset,...,True,{'criteriaParameters': 'not available in demo ...,organic,,,,,not available in demo dataset,,
1,Organic Search,20171016,0643697640977915618,0643697640977915618_1508175522,Not Socially Engaged,1508175522,1,1508175522,NaN,not available in demo dataset,...,,{'criteriaParameters': 'not available in demo ...,organic,,,,,not available in demo dataset,,
2,Organic Search,20171016,6059383810968229466,6059383810968229466_1508143220,Not Socially Engaged,1508143220,1,1508143220,NaN,not available in demo dataset,...,,{'criteriaParameters': 'not available in demo ...,organic,,,,,not available in demo dataset,,
3,Organic Search,20171016,2376720078563423631,2376720078563423631_1508193530,Not Socially Engaged,1508193530,1,1508193530,NaN,not available in demo dataset,...,,{'criteriaParameters': 'not available in demo ...,organic,,,,,not available in demo dataset,,
4,Organic Search,20171016,2314544520795440038,2314544520795440038_1508217442,Not Socially Engaged,1508217442,1,1508217442,NaN,not available in demo dataset,...,,{'criteriaParameters': 'not available in demo ...,organic,,,,,not available in demo dataset,,


In [194]:
del test['trafficSource_adwordsClickInfo']
test.to_csv('test_new_v2.csv', index = False)

# Create a baseline model

In [195]:
train_use.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,transactionRevenue,device_operatingSystemVersion,...,trafficSource_referralPath,trafficSource_isTrueDirect,trafficSource_medium,trafficSource_adwordsClickInfo_slot,trafficSource_adwordsClickInfo_page,trafficSource_adwordsClickInfo_isVideoAd,trafficSource_adwordsClickInfo_adNetworkType,trafficSource_adwordsClickInfo_criteriaParameters,trafficSource_adwordsClickInfo_targetingCriteria,trafficSource_adwordsClickInfo_gclId
752,Direct,20160902,6194193421514403509,6194193421514403509_1472843572,Not Socially Engaged,1472843572,1,1472843572,37860000,not available in demo dataset,...,,True,(none),,,,,not available in demo dataset,,
753,Organic Search,20160902,5327166854580374902,5327166854580374902_1472844906,Not Socially Engaged,1472844906,3,1472844906,306670000,not available in demo dataset,...,,True,organic,,,,,not available in demo dataset,,
799,Referral,20160902,8885051388942907862,8885051388942907862_1472827393,Not Socially Engaged,1472827393,7,1472827393,68030000,not available in demo dataset,...,/,True,referral,,,,,not available in demo dataset,,
802,Referral,20160902,0185467632009737931,0185467632009737931_1472846398,Not Socially Engaged,1472846398,6,1472846398,26250000,not available in demo dataset,...,/,True,referral,,,,,not available in demo dataset,,
859,Referral,20160902,3244885836845029978,3244885836845029978_1472824817,Not Socially Engaged,1472824817,4,1472824817,574150000,not available in demo dataset,...,/,True,referral,,,,,not available in demo dataset,,


In [196]:
test.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,transactionRevenue,device_operatingSystemVersion,...,trafficSource_referralPath,trafficSource_isTrueDirect,trafficSource_medium,trafficSource_adwordsClickInfo_slot,trafficSource_adwordsClickInfo_page,trafficSource_adwordsClickInfo_isVideoAd,trafficSource_adwordsClickInfo_adNetworkType,trafficSource_adwordsClickInfo_criteriaParameters,trafficSource_adwordsClickInfo_targetingCriteria,trafficSource_adwordsClickInfo_gclId
0,Organic Search,20171016,6167871330617112363,6167871330617112363_1508151024,Not Socially Engaged,1508151024,2,1508151024,NaN,not available in demo dataset,...,,True,organic,,,,,not available in demo dataset,,
1,Organic Search,20171016,0643697640977915618,0643697640977915618_1508175522,Not Socially Engaged,1508175522,1,1508175522,NaN,not available in demo dataset,...,,,organic,,,,,not available in demo dataset,,
2,Organic Search,20171016,6059383810968229466,6059383810968229466_1508143220,Not Socially Engaged,1508143220,1,1508143220,NaN,not available in demo dataset,...,,,organic,,,,,not available in demo dataset,,
3,Organic Search,20171016,2376720078563423631,2376720078563423631_1508193530,Not Socially Engaged,1508193530,1,1508193530,NaN,not available in demo dataset,...,,,organic,,,,,not available in demo dataset,,
4,Organic Search,20171016,2314544520795440038,2314544520795440038_1508217442,Not Socially Engaged,1508217442,1,1508217442,NaN,not available in demo dataset,...,,,organic,,,,,not available in demo dataset,,


In [199]:
set(train_use.columns) - set(test.columns)

set()

In [200]:
train_use.shape

(11515, 55)

In [202]:
test.shape

(804684, 55)

In [205]:
train_use['transactionRevenue'].head()

752     37860000
753    306670000
799     68030000
802     26250000
859    574150000
Name: transactionRevenue, dtype: object

In [206]:
test['transactionRevenue'].sum()

0.0

In [233]:
train_test = train_use.append(test)
train_test.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,transactionRevenue,device_operatingSystemVersion,...,trafficSource_referralPath,trafficSource_isTrueDirect,trafficSource_medium,trafficSource_adwordsClickInfo_slot,trafficSource_adwordsClickInfo_page,trafficSource_adwordsClickInfo_isVideoAd,trafficSource_adwordsClickInfo_adNetworkType,trafficSource_adwordsClickInfo_criteriaParameters,trafficSource_adwordsClickInfo_targetingCriteria,trafficSource_adwordsClickInfo_gclId
752,Direct,20160902,6194193421514403509,6194193421514403509_1472843572,Not Socially Engaged,1472843572,1,1472843572,37860000,not available in demo dataset,...,,True,(none),,,,,not available in demo dataset,,
753,Organic Search,20160902,5327166854580374902,5327166854580374902_1472844906,Not Socially Engaged,1472844906,3,1472844906,306670000,not available in demo dataset,...,,True,organic,,,,,not available in demo dataset,,
799,Referral,20160902,8885051388942907862,8885051388942907862_1472827393,Not Socially Engaged,1472827393,7,1472827393,68030000,not available in demo dataset,...,/,True,referral,,,,,not available in demo dataset,,
802,Referral,20160902,0185467632009737931,0185467632009737931_1472846398,Not Socially Engaged,1472846398,6,1472846398,26250000,not available in demo dataset,...,/,True,referral,,,,,not available in demo dataset,,
859,Referral,20160902,3244885836845029978,3244885836845029978_1472824817,Not Socially Engaged,1472824817,4,1472824817,574150000,not available in demo dataset,...,/,True,referral,,,,,not available in demo dataset,,


In [208]:
train_test.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId',
       'socialEngagementType', 'visitId', 'visitNumber', 'visitStartTime',
       'transactionRevenue', 'device_operatingSystemVersion',
       'device_mobileDeviceModel', 'device_mobileInputSelector',
       'device_mobileDeviceInfo', 'device_browserVersion',
       'device_screenColors', 'device_browserSize', 'device_browser',
       'device_isMobile', 'device_mobileDeviceMarketingName',
       'device_screenResolution', 'device_operatingSystem',
       'device_mobileDeviceBranding', 'device_language',
       'device_deviceCategory', 'device_flashVersion', 'geoNetwork_country',
       'geoNetwork_city', 'geoNetwork_longitude', 'geoNetwork_latitude',
       'geoNetwork_networkLocation', 'geoNetwork_continent',
       'geoNetwork_metro', 'geoNetwork_subContinent', 'geoNetwork_cityId',
       'geoNetwork_networkDomain', 'geoNetwork_region', 'totals_pageviews',
       'totals_hits', 'totals_newVisits', 'totals_transactionRev

In [211]:
train_test['channelGrouping'].value_counts()

Organic Search    360840
Direct            132150
Social            128958
Referral          111780
Display            45163
Paid Search        20769
Affiliates         16521
(Other)               18
Name: channelGrouping, dtype: int64

In [234]:
train_test['date'] = pd.to_datetime(train_test['visitStartTime'], unit='s')
train_test['sess_date_dow'] = train_test['date'].dt.dayofweek
train_test['sess_date_hours'] = train_test['date'].dt.hour
train_test['sess_date_dom'] = train_test['date'].dt.day

In [235]:
# https://www.kaggle.com/prashantkikani/teach-lightgbm-to-sum-predictions-fe
def browser_mapping(x):
    browsers = ['chrome','safari','firefox','internet explorer','edge','opera','coc coc','maxthon','iron']
    if x in browsers:
        return x.lower()
    elif  ('android' in x) or ('samsung' in x) or ('mini' in x) or ('iphone' in x) or ('in-app' in x) or ('playstation' in x):
        return 'mobile browser'
    elif  ('mozilla' in x) or ('chrome' in x) or ('blackberry' in x) or ('nokia' in x) or ('browser' in x) or ('amazon' in x):
        return 'mobile browser'
    elif  ('lunascape' in x) or ('netscape' in x) or ('blackberry' in x) or ('konqueror' in x) or ('puffin' in x) or ('amazon' in x):
        return 'mobile browser'
    elif '(not set)' in x:
        return x
    else:
        return 'others'

In [236]:
train_test['device_browser'] = train_test['device_browser'].map(lambda x:browser_mapping(str(x).lower())).astype('str')

In [237]:
def adcontents_mapping(x):
    if  ('google' in x):
        return 'google'
    elif  ('placement' in x) | ('placememnt' in x):
        return 'placement'
    elif '(not set)' in x or 'nan' in x:
        return x
    elif 'ad' in x:
        return 'ad'
    else:
        return 'others'

In [238]:
train_test['trafficSource_adContent'] = train_test['trafficSource_adContent'].map(lambda x:adcontents_mapping(str(x).lower())).astype('str')
train_test['trafficSource_adContent'].value_counts()

others       762831
google        43226
placement      5157
ad             4985
Name: trafficSource_adContent, dtype: int64

In [239]:
def source_mapping(x):
    if  ('google' in x):
        return 'google'
    elif  ('youtube' in x):
        return 'youtube'
    elif '(not set)' in x or 'nan' in x:
        return x
    elif 'yahoo' in x:
        return 'yahoo'
    elif 'facebook' in x:
        return 'facebook'
    elif 'reddit' in x:
        return 'reddit'
    elif 'bing' in x:
        return 'bing'
    elif 'quora' in x:
        return 'quora'
    elif 'outlook' in x:
        return 'outlook'
    elif 'linkedin' in x:
        return 'linkedin'
    elif 'pinterest' in x:
        return 'pinterest'
    elif 'ask' in x:
        return 'ask'
    elif 'siliconvalley' in x:
        return 'siliconvalley'
    elif 'lunametrics' in x:
        return 'lunametrics'
    elif 'amazon' in x:
        return 'amazon'
    elif 'mysearch' in x:
        return 'mysearch'
    elif 'qiita' in x:
        return 'qiita'
    elif 'messenger' in x:
        return 'messenger'
    elif 'twitter' in x:
        return 'twitter'
    elif 't.co' in x:
        return 't.co'
    elif 'vk.com' in x:
        return 'vk.com'
    elif 'search' in x:
        return 'search'
    elif 'edu' in x:
        return 'edu'
    elif 'mail' in x:
        return 'mail'
    elif 'ad' in x:
        return 'ad'
    elif 'golang' in x:
        return 'golang'
    elif 'direct' in x:
        return 'direct'
    elif 'dealspotr' in x:
        return 'dealspotr'
    elif 'sashihara' in x:
        return 'sashihara'
    elif 'phandroid' in x:
        return 'phandroid'
    elif 'baidu' in x:
        return 'baidu'
    elif 'mdn' in x:
        return 'mdn'
    elif 'duckduckgo' in x:
        return 'duckduckgo'
    elif 'seroundtable' in x:
        return 'seroundtable'
    elif 'metrics' in x:
        return 'metrics'
    elif 'sogou' in x:
        return 'sogou'
    elif 'businessinsider' in x:
        return 'businessinsider'
    elif 'github' in x:
        return 'github'
    elif 'gophergala' in x:
        return 'gophergala'
    elif 'yandex' in x:
        return 'yandex'
    elif 'msn' in x:
        return 'msn'
    elif 'dfa' in x:
        return 'dfa'
    elif '(not set)' in x:
        return '(not set)'
    elif 'feedly' in x:
        return 'feedly'
    elif 'arstechnica' in x:
        return 'arstechnica'
    elif 'squishable' in x:
        return 'squishable'
    elif 'flipboard' in x:
        return 'flipboard'
    elif 't-online.de' in x:
        return 't-online.de'
    elif 'sm.cn' in x:
        return 'sm.cn'
    elif 'wow' in x:
        return 'wow'
    elif 'baidu' in x:
        return 'baidu'
    elif 'partners' in x:
        return 'partners'
    else:
        return 'others'

In [240]:
train_test['trafficSource_source'] = train_test['trafficSource_source'].map(lambda x:source_mapping(str(x).lower())).astype('str')
train_test['trafficSource_source']

752         direct
753         google
799         google
802         google
859         google
866         google
893         direct
910         google
922         google
925         google
930         google
942         google
971         google
974         google
976         google
999         direct
1000        google
1008        google
1020        google
1022        google
1026        google
1027        google
1037        google
1041        google
1054        google
1055        google
1056        google
2774           dfa
3161        google
3268        google
            ...   
804654     youtube
804655     youtube
804656     youtube
804657     youtube
804658     youtube
804659     youtube
804660     youtube
804661     youtube
804662     youtube
804663     youtube
804664     youtube
804665     youtube
804666     youtube
804667     youtube
804668     youtube
804669     youtube
804670     youtube
804671     youtube
804672     youtube
804673     youtube
804674     youtube
804675     y

In [241]:
train_test['trafficSource_source'].value_counts()

google             530144
direct             132166
youtube            116886
partners            16529
facebook             5562
reddit               2082
bing                 1863
baidu                1689
qiita                1581
quora                1487
yahoo                1230
dfa                  1141
golang                640
vk.com                596
t.co                  489
ask                   289
dealspotr             262
lunametrics           222
duckduckgo            221
sashihara             147
edu                   145
linkedin               99
phandroid              99
pinterest              75
search                 70
messenger              69
metrics                67
others                 63
outlook                62
yandex                 28
mdn                    23
github                 22
msn                    21
mysearch               21
gophergala             19
seroundtable           15
arstechnica            12
t-online.de            11
mail        

In [244]:
train_test['source_country'] = train_test['trafficSource_source'] + '_' + train_test['geoNetwork_country']

In [248]:
train_test['browser_category'] = train_test['device_browser'] + '_' + train_test['device_deviceCategory']
train_test['browser_category'].value_counts()

chrome_desktop               425175
chrome_mobile                127176
safari_mobile                 75055
mobile browser_mobile         58875
safari_desktop                37170
firefox_desktop               25396
safari_tablet                 18475
internet explorer_desktop     14495
chrome_tablet                 10694
edge_desktop                  10205
mobile browser_tablet          4017
opera_desktop                  3159
internet explorer_tablet       1518
mobile browser_desktop         1356
firefox_mobile                 1169
opera_mobile                    766
coc coc_desktop                 499
firefox_tablet                  402
internet explorer_mobile        195
edge_mobile                     191
others_desktop                   91
maxthon_desktop                  38
others_mobile                    26
opera_tablet                     22
coc coc_tablet                    9
iron_desktop                      7
coc coc_mobile                    7
others_tablet               

In [250]:
train_test['browser_os'] = train_test['device_browser'] + '_' + train_test['device_operatingSystem']

In [252]:
train_test['device_deviceCategory_channelGrouping'] = train_test['device_deviceCategory'] + "_" + train_test['channelGrouping']

In [255]:
train_test['channelGrouping_browser'] = train_test['device_browser'] + "_" + train_test['channelGrouping']
train_test['channelGrouping_browser']

752               chrome_Direct
753       chrome_Organic Search
799             chrome_Referral
802             chrome_Referral
859             chrome_Referral
866             chrome_Referral
893               safari_Direct
910             chrome_Referral
922       chrome_Organic Search
925             chrome_Referral
930       chrome_Organic Search
942             chrome_Referral
971             chrome_Referral
974       safari_Organic Search
976             chrome_Referral
999               safari_Direct
1000      chrome_Organic Search
1008            chrome_Referral
1020      chrome_Organic Search
1022            chrome_Referral
1026            chrome_Referral
1027            chrome_Referral
1037            chrome_Referral
1041            chrome_Referral
1054            chrome_Referral
1055      chrome_Organic Search
1056            chrome_Referral
2774             chrome_Display
3161            chrome_Referral
3268            chrome_Referral
                  ...          
804654  

In [257]:
train_test['channelGrouping_OS'] = train_test['device_operatingSystem'] + "_" + train_test['channelGrouping']

In [259]:
train_test['content_source'] = train_test['trafficSource_adContent'] + "_" + train_test['source_country']

In [261]:
train_test['medium_source'] = train_test['trafficSource_medium'] + "_" + train_test['source_country']

In [262]:
for i in ['geoNetwork_city', 'geoNetwork_continent', 'geoNetwork_country','geoNetwork_metro', 'geoNetwork_networkDomain', 'geoNetwork_region','geoNetwork_subContinent']:
    for j in ['device_browser','device_deviceCategory', 'device_operatingSystem', 'trafficSource_source']:
        train_test[i + "_" + j] = train_test[i] + "_" + train_test[j]

In [263]:
train_test.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,transactionRevenue,device_operatingSystemVersion,...,geoNetwork_networkDomain_device_operatingSystem,geoNetwork_networkDomain_trafficSource_source,geoNetwork_region_device_browser,geoNetwork_region_device_deviceCategory,geoNetwork_region_device_operatingSystem,geoNetwork_region_trafficSource_source,geoNetwork_subContinent_device_browser,geoNetwork_subContinent_device_deviceCategory,geoNetwork_subContinent_device_operatingSystem,geoNetwork_subContinent_trafficSource_source
752,Direct,2016-09-02 19:12:52,6194193421514403509,6194193421514403509_1472843572,Not Socially Engaged,1472843572,1,1472843572,37860000,not available in demo dataset,...,(not set)_Linux,(not set)_direct,Michigan_chrome,Michigan_desktop,Michigan_Linux,Michigan_direct,Northern America_chrome,Northern America_desktop,Northern America_Linux,Northern America_direct
753,Organic Search,2016-09-02 19:35:06,5327166854580374902,5327166854580374902_1472844906,Not Socially Engaged,1472844906,3,1472844906,306670000,not available in demo dataset,...,(not set)_Macintosh,(not set)_google,New York_chrome,New York_desktop,New York_Macintosh,New York_google,Northern America_chrome,Northern America_desktop,Northern America_Macintosh,Northern America_google
799,Referral,2016-09-02 14:43:13,8885051388942907862,8885051388942907862_1472827393,Not Socially Engaged,1472827393,7,1472827393,68030000,not available in demo dataset,...,(not set)_Linux,(not set)_google,New York_chrome,New York_desktop,New York_Linux,New York_google,Northern America_chrome,Northern America_desktop,Northern America_Linux,Northern America_google
802,Referral,2016-09-02 19:59:58,0185467632009737931,0185467632009737931_1472846398,Not Socially Engaged,1472846398,6,1472846398,26250000,not available in demo dataset,...,(not set)_Windows,(not set)_google,California_chrome,California_desktop,California_Windows,California_google,Northern America_chrome,Northern America_desktop,Northern America_Windows,Northern America_google
859,Referral,2016-09-02 14:00:17,3244885836845029978,3244885836845029978_1472824817,Not Socially Engaged,1472824817,4,1472824817,574150000,not available in demo dataset,...,(not set)_Macintosh,(not set)_google,not available in demo dataset_chrome,not available in demo dataset_desktop,not available in demo dataset_Macintosh,not available in demo dataset_google,Northern America_chrome,Northern America_desktop,Northern America_Macintosh,Northern America_google


In [264]:
train_test.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId',
       'socialEngagementType', 'visitId', 'visitNumber', 'visitStartTime',
       'transactionRevenue', 'device_operatingSystemVersion',
       'device_mobileDeviceModel', 'device_mobileInputSelector',
       'device_mobileDeviceInfo', 'device_browserVersion',
       'device_screenColors', 'device_browserSize', 'device_browser',
       'device_isMobile', 'device_mobileDeviceMarketingName',
       'device_screenResolution', 'device_operatingSystem',
       'device_mobileDeviceBranding', 'device_language',
       'device_deviceCategory', 'device_flashVersion', 'geoNetwork_country',
       'geoNetwork_city', 'geoNetwork_longitude', 'geoNetwork_latitude',
       'geoNetwork_networkLocation', 'geoNetwork_continent',
       'geoNetwork_metro', 'geoNetwork_subContinent', 'geoNetwork_cityId',
       'geoNetwork_networkDomain', 'geoNetwork_region', 'totals_pageviews',
       'totals_hits', 'totals_newVisits', 'totals_transactionRev

In [265]:
train_test['channelGrouping'].value_counts()

Organic Search    360840
Direct            132150
Social            128958
Referral          111780
Display            45163
Paid Search        20769
Affiliates         16521
(Other)               18
Name: channelGrouping, dtype: int64

In [266]:
train_test['socialEngagementType'].value_counts()

Not Socially Engaged    816199
Name: socialEngagementType, dtype: int64

In [268]:
pd.isnull(train_test['socialEngagementType']).value_counts()

False    816199
Name: socialEngagementType, dtype: int64

In [274]:
train_test['device_operatingSystemVersion'].value_counts()

not available in demo dataset    816199
Name: device_operatingSystemVersion, dtype: int64

In [275]:
train_test['device_mobileDeviceModel'].value_counts()

not available in demo dataset    816199
Name: device_mobileDeviceModel, dtype: int64

In [278]:
train_test['device_browserVersion'].value_counts()

not available in demo dataset    816199
Name: device_browserVersion, dtype: int64

In [280]:
cols = ['device_screenColors', 'device_browserSize', 'device_browser',
       'device_isMobile', 'device_mobileDeviceMarketingName',
       'device_screenResolution', 'device_operatingSystem',
       'device_mobileDeviceBranding', 'device_language',
       'device_deviceCategory', 'device_flashVersion', 'geoNetwork_country',
       'geoNetwork_city', 'geoNetwork_longitude', 'geoNetwork_latitude',
       'geoNetwork_networkLocation', 'geoNetwork_continent',
       'geoNetwork_metro', 'geoNetwork_subContinent', 'geoNetwork_cityId',
       'geoNetwork_networkDomain', 'geoNetwork_region', 'totals_pageviews',
       'totals_hits', 'totals_newVisits', 'totals_transactionRevenue',
       'totals_visits', 'trafficSource_adContent', 'trafficSource_campaign',
       'trafficSource_source', 'trafficSource_keyword',
       'trafficSource_referralPath', 'trafficSource_isTrueDirect',
       'trafficSource_medium', 'trafficSource_adwordsClickInfo_slot',
       'trafficSource_adwordsClickInfo_page',
       'trafficSource_adwordsClickInfo_isVideoAd',
       'trafficSource_adwordsClickInfo_adNetworkType',
       'trafficSource_adwordsClickInfo_criteriaParameters',
       'trafficSource_adwordsClickInfo_targetingCriteria']
for cl in cols:
    print(cl)
    print(train_test[cl].value_counts())
    print('\n')

device_screenColors
not available in demo dataset    816199
Name: device_screenColors, dtype: int64


device_browserSize
not available in demo dataset    816199
Name: device_browserSize, dtype: int64


device_browser
chrome               563045
safari               130700
mobile browser        64248
firefox               26967
internet explorer     16208
edge                  10396
opera                  3947
coc coc                 515
others                  121
maxthon                  41
iron                      7
(not set)                 4
Name: device_browser, dtype: int64


device_isMobile
False    517635
True     298564
Name: device_isMobile, dtype: int64


device_mobileDeviceMarketingName
not available in demo dataset    816199
Name: device_mobileDeviceMarketingName, dtype: int64


device_screenResolution
not available in demo dataset    816199
Name: device_screenResolution, dtype: int64


device_operatingSystem
Windows             271957
Macintosh           191002
Android  

geoNetwork_subContinent
Northern America      388831
Southern Asia          61752
Western Europe         56069
Northern Europe        53552
Eastern Asia           44212
Southeast Asia         43866
South America          33479
Southern Europe        31412
Eastern Europe         28772
Western Asia           22544
Central America        14007
Australasia            12988
Northern Africa         9573
Western Africa          4515
Southern Africa         3852
Eastern Africa          2270
Caribbean               1992
(not set)               1055
Central Asia             772
Middle Africa            534
Micronesian Region        73
Melanesia                 52
Polynesia                 27
Name: geoNetwork_subContinent, dtype: int64


geoNetwork_cityId
not available in demo dataset    816199
Name: geoNetwork_cityId, dtype: int64


geoNetwork_networkDomain
(not set)                   260813
unknown.unknown             124162
comcast.net                  27677
rr.com                       14252


Name: totals_transactionRevenue, Length: 5333, dtype: int64


totals_visits
1    816199
Name: totals_visits, dtype: int64


trafficSource_adContent
others       762831
google        43226
placement      5157
ad             4985
Name: trafficSource_adContent, dtype: int64


trafficSource_campaign
(not set)                                                               739977
1000557 | GA | US | en | Hybrid | GDN Text+Banner | AS                   24410
Data Share Promo                                                         16520
1000557 | GA | US | en | Hybrid | GDN Remarketing                        15149
AW - Dynamic Search Ads Whole Site                                        9256
AW - Accessories                                                          2775
Smart Display Campaign                                                    2664
"google + redesign/Accessories March 17" All Users Similar Audiences      1179
Page: contains "/google+redesign/drinkware" Similar Audiences          

trafficSource_isTrueDirect
        548682
True    267517
Name: trafficSource_isTrueDirect, dtype: int64


trafficSource_medium
organic      360840
referral     240738
(none)       132150
cpc           62992
affiliate     16521
cpm            2940
(not set)        18
Name: trafficSource_medium, dtype: int64


trafficSource_adwordsClickInfo_slot
                          761931
RHS                        42251
Top                        11940
Google Display Network        77
Name: trafficSource_adwordsClickInfo_slot, dtype: int64


trafficSource_adwordsClickInfo_page
      761931
1      53005
2        984
3        162
4         78
5         23
6         10
7          3
9          1
12         1
8          1
Name: trafficSource_adwordsClickInfo_page, dtype: int64


trafficSource_adwordsClickInfo_isVideoAd
         761931
False     54268
Name: trafficSource_adwordsClickInfo_isVideoAd, dtype: int64


trafficSource_adwordsClickInfo_adNetworkType
                   761931
Content             

TypeError: unhashable type: 'dict'

In [305]:
features_to_exclude = ['date', 'fullVisitorId', 'sessionId', 'visitId', 'visitStartTime']
features_with_static_values = ['socialEngagementType','device_operatingSystemVersion','device_mobileDeviceModel','device_mobileInputSelector', 'device_mobileDeviceInfo', 'device_browserVersion', 'device_screenColors', 'device_browserSize', 'device_mobileDeviceMarketingName', 'device_screenResolution', 'device_mobileDeviceBranding', 'device_language', 'device_flashVersion', 'geoNetwork_longitude', 'geoNetwork_latitude', 'geoNetwork_networkLocation', 'geoNetwork_cityId','totals_visits', 'trafficSource_adwordsClickInfo_criteriaParameters','trafficSource_adwordsClickInfo_targetingCriteria']
target = 'totals_transactionRevenue'

In [306]:
features = list(set(train_test.columns) - set(features_to_exclude) - set(features_with_static_values) - set([target]))
features

['trafficSource_adContent',
 'device_browser',
 'totals_hits',
 'trafficSource_referralPath',
 'trafficSource_isTrueDirect',
 'totals_pageviews',
 'totals_newVisits',
 'geoNetwork_metro_device_deviceCategory',
 'medium_source',
 'sess_date_dow',
 'trafficSource_adwordsClickInfo_isVideoAd',
 'geoNetwork_networkDomain_trafficSource_source',
 'geoNetwork_region',
 'geoNetwork_networkDomain',
 'trafficSource_adwordsClickInfo_slot',
 'trafficSource_adwordsClickInfo_adNetworkType',
 'device_operatingSystem',
 'device_deviceCategory',
 'geoNetwork_region_device_operatingSystem',
 'geoNetwork_country_device_browser',
 'geoNetwork_subContinent',
 'geoNetwork_region_device_deviceCategory',
 'geoNetwork_country',
 'geoNetwork_city',
 'content_source',
 'geoNetwork_networkDomain_device_deviceCategory',
 'visitNumber',
 'geoNetwork_continent_device_operatingSystem',
 'geoNetwork_networkDomain_device_operatingSystem',
 'trafficSource_campaign',
 'sess_date_hours',
 'channelGrouping_browser',
 'chann

In [288]:
for i in features:
    if train_test[i].dtype == 'object':
        print(i)
        print(train_test[i].nunique())
        print('\n')

trafficSource_adContent
4


device_browser
12


totals_hits
262


trafficSource_referralPath
2217


trafficSource_isTrueDirect
2


totals_pageviews
194


totals_newVisits
2


geoNetwork_metro_device_deviceCategory
275


medium_source
2154


trafficSource_adwordsClickInfo_isVideoAd
2


geoNetwork_networkDomain_trafficSource_source
40325


geoNetwork_region
378


geoNetwork_networkDomain
25889


trafficSource_adwordsClickInfo_slot
4


trafficSource_adwordsClickInfo_adNetworkType
4


device_operatingSystem
22


device_deviceCategory
3


geoNetwork_region_device_operatingSystem
1731


geoNetwork_country_device_browser
1233


geoNetwork_subContinent
23


geoNetwork_region_device_deviceCategory
891


geoNetwork_country
220


geoNetwork_city
744


content_source
2023


geoNetwork_networkDomain_device_deviceCategory
32174


totals_transactionRevenue
5333


geoNetwork_continent_device_operatingSystem
85


geoNetwork_networkDomain_device_operatingSystem
39547


trafficSource_campaign
33


channe

In [318]:
other_features_to_exclude = 'trafficSource_referralPath'
features.remove(other_features_to_exclude)

In [295]:
train_test['totals_hits'] = pd.to_numeric(pd.Series(train_test['totals_hits']),errors='coerce')

In [298]:
train_test['totals_pageviews'] = pd.to_numeric(pd.Series(train_test['totals_pageviews']),errors='coerce')

In [320]:
#numeric features
for i in features:
    if train_test[i].dtype != 'object':
        print(i)

totals_hits
totals_pageviews
sess_date_dow
visitNumber
sess_date_hours
sess_date_dom
device_isMobile


In [321]:
#One hot encode
for i in features:
    if train_test[i].dtype == 'object' and train_test[i].nunique()<=15:
        print(i)

trafficSource_adContent
device_browser
trafficSource_isTrueDirect
totals_newVisits
trafficSource_adwordsClickInfo_isVideoAd
trafficSource_adwordsClickInfo_slot
trafficSource_adwordsClickInfo_adNetworkType
device_deviceCategory
channelGrouping
geoNetwork_continent
trafficSource_adwordsClickInfo_page
trafficSource_medium


In [322]:
#Label encode
for i in features:
    if train_test[i].dtype == 'object' and train_test[i].nunique()>15:
        print(i)

geoNetwork_metro_device_deviceCategory
medium_source
geoNetwork_networkDomain_trafficSource_source
geoNetwork_region
geoNetwork_networkDomain
device_operatingSystem
geoNetwork_region_device_operatingSystem
geoNetwork_country_device_browser
geoNetwork_subContinent
geoNetwork_region_device_deviceCategory
geoNetwork_country
geoNetwork_city
content_source
geoNetwork_networkDomain_device_deviceCategory
geoNetwork_continent_device_operatingSystem
geoNetwork_networkDomain_device_operatingSystem
trafficSource_campaign
channelGrouping_browser
channelGrouping_OS
geoNetwork_metro_device_browser
geoNetwork_region_trafficSource_source
geoNetwork_metro_trafficSource_source
geoNetwork_country_device_deviceCategory
geoNetwork_region_device_browser
geoNetwork_city_device_deviceCategory
geoNetwork_continent_device_deviceCategory
geoNetwork_subContinent_device_deviceCategory
transactionRevenue
geoNetwork_subContinent_device_operatingSystem
source_country
trafficSource_keyword
geoNetwork_city_trafficSourc

# Label Encode

In [325]:
le_features = ['geoNetwork_metro_device_deviceCategory','medium_source','geoNetwork_networkDomain_trafficSource_source','geoNetwork_region','geoNetwork_networkDomain','device_operatingSystem','geoNetwork_region_device_operatingSystem','geoNetwork_country_device_browser','geoNetwork_subContinent','geoNetwork_region_device_deviceCategory','geoNetwork_country','geoNetwork_city','content_source','geoNetwork_networkDomain_device_deviceCategory','geoNetwork_continent_device_operatingSystem','geoNetwork_networkDomain_device_operatingSystem','trafficSource_campaign','channelGrouping_browser','channelGrouping_OS','geoNetwork_metro_device_browser','geoNetwork_region_trafficSource_source','geoNetwork_metro_trafficSource_source','geoNetwork_country_device_deviceCategory','geoNetwork_region_device_browser','geoNetwork_city_device_deviceCategory','geoNetwork_continent_device_deviceCategory','geoNetwork_subContinent_device_deviceCategory','transactionRevenue','geoNetwork_subContinent_device_operatingSystem','source_country','trafficSource_keyword','geoNetwork_city_trafficSource_source','browser_os','geoNetwork_metro','geoNetwork_continent_device_browser','geoNetwork_metro_device_operatingSystem','device_deviceCategory_channelGrouping','browser_category','geoNetwork_networkDomain_device_browser','trafficSource_adwordsClickInfo_gclId','geoNetwork_subContinent_trafficSource_source','geoNetwork_country_device_operatingSystem','trafficSource_source','geoNetwork_continent_trafficSource_source','geoNetwork_country_trafficSource_source','geoNetwork_subContinent_device_browser','geoNetwork_city_device_browser','geoNetwork_city_device_operatingSystem']

In [332]:
for f in le_features:#Add all categorical features in the list
    lbl = LabelEncoder()
    lbl.fit(list(train_test[f].values))
    train_test[f] = lbl.transform(list(train_test[f].values))

# One Hot Encode

In [324]:
ohe_features = ['trafficSource_adContent','device_browser','trafficSource_isTrueDirect','totals_newVisits','trafficSource_adwordsClickInfo_isVideoAd','trafficSource_adwordsClickInfo_slot','trafficSource_adwordsClickInfo_adNetworkType','device_deviceCategory','channelGrouping','geoNetwork_continent','trafficSource_adwordsClickInfo_page','trafficSource_medium']

In [328]:
one_hot_encoded_train_test = pd.get_dummies(train_test[ohe_features])    

In [345]:
train_test_new = pd.concat([train_test,one_hot_encoded_train_test], axis=1)

In [346]:
train_test_new.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,transactionRevenue,device_operatingSystemVersion,...,trafficSource_adwordsClickInfo_page_7,trafficSource_adwordsClickInfo_page_8,trafficSource_adwordsClickInfo_page_9,trafficSource_medium_(none),trafficSource_medium_(not set),trafficSource_medium_affiliate,trafficSource_medium_cpc,trafficSource_medium_cpm,trafficSource_medium_organic,trafficSource_medium_referral
752,Direct,2016-09-02 19:12:52,6194193421514403509,6194193421514403509_1472843572,Not Socially Engaged,1472843572,1,1472843572,2967,not available in demo dataset,...,0,0,0,1,0,0,0,0,0,0
753,Organic Search,2016-09-02 19:35:06,5327166854580374902,5327166854580374902_1472844906,Not Socially Engaged,1472844906,3,1472844906,2510,not available in demo dataset,...,0,0,0,0,0,0,0,0,1,0
799,Referral,2016-09-02 14:43:13,8885051388942907862,8885051388942907862_1472827393,Not Socially Engaged,1472827393,7,1472827393,4357,not available in demo dataset,...,0,0,0,0,0,0,0,0,0,1
802,Referral,2016-09-02 19:59:58,0185467632009737931,0185467632009737931_1472846398,Not Socially Engaged,1472846398,6,1472846398,2173,not available in demo dataset,...,0,0,0,0,0,0,0,0,0,1
859,Referral,2016-09-02 14:00:17,3244885836845029978,3244885836845029978_1472824817,Not Socially Engaged,1472824817,4,1472824817,3933,not available in demo dataset,...,0,0,0,0,0,0,0,0,0,1


In [395]:
features_use = list(set(train_test_new.columns) - set(ohe_features) - set(le_features) - set(features_to_exclude) -set(features_with_static_values) - set([target]) - set(['trafficSource_referralPath']))

In [396]:
features_use

['geoNetwork_continent_Asia',
 'trafficSource_medium_(not set)',
 'trafficSource_adwordsClickInfo_page_',
 'totals_newVisits_1',
 'device_browser_others',
 'trafficSource_adwordsClickInfo_adNetworkType_Search partners',
 'totals_hits',
 'totals_pageviews',
 'trafficSource_adContent_others',
 'trafficSource_adwordsClickInfo_isVideoAd_',
 'channelGrouping_Paid Search',
 'sess_date_dow',
 'device_browser_chrome',
 'trafficSource_adwordsClickInfo_page_2',
 'device_deviceCategory_tablet',
 'geoNetwork_continent_Africa',
 'device_deviceCategory_mobile',
 'geoNetwork_continent_(not set)',
 'device_browser_mobile browser',
 'trafficSource_medium_affiliate',
 'device_browser_(not set)',
 'device_browser_maxthon',
 'trafficSource_adwordsClickInfo_adNetworkType_',
 'trafficSource_medium_cpc',
 'trafficSource_adContent_ad',
 'trafficSource_adwordsClickInfo_page_5',
 'channelGrouping_Referral',
 'trafficSource_adwordsClickInfo_slot_RHS',
 'trafficSource_adwordsClickInfo_slot_Top',
 'channelGrouping

In [376]:
train_test_new[target] = pd.to_numeric(pd.Series(train_test_new[target]),errors='coerce')

In [378]:
train_test_new[features_use].head()

,geoNetwork_continent_Asia,trafficSource_medium_(not set),trafficSource_adwordsClickInfo_page_,totals_newVisits_1,device_browser_others,trafficSource_adwordsClickInfo_adNetworkType_Search partners,totals_hits,trafficSource_referralPath,totals_pageviews,trafficSource_adContent_others,...,trafficSource_isTrueDirect_True,device_deviceCategory_desktop,trafficSource_adwordsClickInfo_page_6,device_isMobile,trafficSource_adwordsClickInfo_page_12,trafficSource_adwordsClickInfo_isVideoAd_False,trafficSource_adwordsClickInfo_slot_,trafficSource_adwordsClickInfo_slot_Google Display Network,trafficSource_adwordsClickInfo_page_9,device_browser_coc coc
752,0,0,1,1,0,0,11,,11.0,1,...,1,1,0,False,0,0,1,0,0,0
753,0,0,1,0,0,0,11,,10.0,1,...,1,1,0,False,0,0,1,0,0,0
799,0,0,1,0,0,0,13,/,11.0,1,...,1,1,0,False,0,0,1,0,0,0
802,0,0,1,0,0,0,13,/,12.0,1,...,1,1,0,False,0,0,1,0,0,0
859,0,0,1,0,0,0,17,/,14.0,1,...,1,1,0,False,0,0,1,0,0,0


In [377]:
for ft in features_use:
    if train_test_new[ft].dtype == 'O':
        print(ft)

trafficSource_referralPath


In [384]:
train_test_new.to_csv('train_test_new.csv', index = False)

In [409]:
train_all = train_test_new[0:len(train_use.index)]
test_xgb  = train_test_new[len(train_use.index):len(train_test_new.index)]

train_ids = random.sample(list(train_all.index),int(.8*len(train_all.index)))
train_xgb = train_all[train_all.index.isin(train_ids)]
valid_xgb = train_all[~train_all.index.isin(train_ids)]

In [410]:
target in features_use

False

In [411]:
dtrain = xgb.DMatrix(train_xgb[features_use], train_xgb[target] , missing=np.nan)
dvalid = xgb.DMatrix(valid_xgb[features_use] , missing=np.nan)
dtest = xgb.DMatrix(test_xgb[features_use], missing=np.nan)

In [414]:
nrounds = 6000
watchlist = [(dtrain, 'train')]
params = {"objective": "reg:linear","nthread": 4,"silent": 1,"eta": 0.05, "max_depth": 5, "subsample": 0.9, "colsample_bytree": 0.7,"min_child_weight": 1,"seed": 2016}
bst = xgb.train(params, dtrain, num_boost_round=nrounds, evals=watchlist, verbose_eval=20)

[0]	train-rmse:4.12523e+08
[20]	train-rmse:3.217e+08
[40]	train-rmse:2.79422e+08
[60]	train-rmse:2.58187e+08
[80]	train-rmse:2.45587e+08
[100]	train-rmse:2.37984e+08
[120]	train-rmse:2.33698e+08
[140]	train-rmse:2.28822e+08
[160]	train-rmse:2.239e+08
[180]	train-rmse:2.20216e+08
[200]	train-rmse:2.16619e+08
[220]	train-rmse:2.13767e+08
[240]	train-rmse:2.10449e+08
[260]	train-rmse:2.07393e+08
[280]	train-rmse:2.04057e+08
[300]	train-rmse:2.01429e+08
[320]	train-rmse:1.98861e+08
[340]	train-rmse:1.96035e+08
[360]	train-rmse:1.93618e+08
[380]	train-rmse:1.91827e+08
[400]	train-rmse:1.89296e+08
[420]	train-rmse:1.87568e+08
[440]	train-rmse:1.85383e+08
[460]	train-rmse:1.8293e+08
[480]	train-rmse:1.81252e+08
[500]	train-rmse:1.79506e+08
[520]	train-rmse:1.77886e+08
[540]	train-rmse:1.76404e+08
[560]	train-rmse:1.74022e+08
[580]	train-rmse:1.72628e+08
[600]	train-rmse:1.70736e+08
[620]	train-rmse:1.68605e+08
[640]	train-rmse:1.67318e+08
[660]	train-rmse:1.65797e+08
[680]	train-rmse:1.64399e

[5540]	train-rmse:5.68953e+07
[5560]	train-rmse:5.67431e+07
[5580]	train-rmse:5.65533e+07
[5600]	train-rmse:5.64141e+07
[5620]	train-rmse:5.62561e+07
[5640]	train-rmse:5.61145e+07
[5660]	train-rmse:5.59436e+07
[5680]	train-rmse:5.57901e+07
[5700]	train-rmse:5.56516e+07
[5720]	train-rmse:5.54929e+07
[5740]	train-rmse:5.53338e+07
[5760]	train-rmse:5.52031e+07
[5780]	train-rmse:5.50469e+07
[5800]	train-rmse:5.49235e+07
[5820]	train-rmse:5.47919e+07
[5840]	train-rmse:5.46169e+07
[5860]	train-rmse:5.44561e+07
[5880]	train-rmse:5.43128e+07
[5900]	train-rmse:5.40768e+07
[5920]	train-rmse:5.39191e+07
[5940]	train-rmse:5.37597e+07
[5960]	train-rmse:5.36246e+07
[5980]	train-rmse:5.3491e+07
[5999]	train-rmse:5.33649e+07


In [416]:
pickle.dump(bst, open("xgb1.pickle.dat", "wb"))

In [472]:
valid_preds = [i if i>0 else 0 for i in list(bst.predict(dvalid))]

In [475]:
for_valid = pd.DataFrame({'fullVisitorId' : valid_xgb['fullVisitorId'],'PredictedLogRevenue' : valid_preds, 'target' : valid_xgb[target]})

In [476]:
for_valid.head()

,PredictedLogRevenue,fullVisitorId,target
799,32618470.0,8885051388942907862,68030000.0
925,11095814.0,6664733704830724714,35080000.0
1020,77567608.0,6712520540378436905,33670000.0
1037,0.0,8585031285178547027,5150000.0
1041,170394144.0,3741689406060673010,90850000.0


In [477]:
for_valid_new = for_valid.groupby('fullVisitorId')[['PredictedLogRevenue','target']].sum().reset_index()

In [478]:
for_valid_new.head()

,fullVisitorId,PredictedLogRevenue,target
0,1123528056036404,256637600.0,96960000.0
1,3450834640354121,279776256.0,57950000.0
2,5339642194263188,13752170.0,9470000.0
3,8803033727416939,0.0,18990000.0
4,10295111715775250,136565440.0,2390000.0


In [480]:
for_valid_new['PredictedLogRevenue'] = for_valid_new['PredictedLogRevenue'].apply(lambda x : math.log(x) if x >0 else 0)

In [483]:
for_valid_new['TargetLogRevenue'] = for_valid_new['target'].apply(lambda x : math.log(x) if x >0 else 0)

In [484]:
for_valid_new.head()

,fullVisitorId,PredictedLogRevenue,target,TargetLogRevenue
0,1123528056036404,19.363176,96960000.0,18.389809
1,3450834640354121,19.449501,57950000.0,17.875091
2,5339642194263188,16.436707,9470000.0,16.063639
3,8803033727416939,0.000000,18990000.0,16.759423
4,10295111715775250,18.732314,2390000.0,14.686804


In [486]:
rms = sqrt(mean_squared_error(for_valid_new['TargetLogRevenue'], for_valid_new['PredictedLogRevenue']))
print(rms)

5.559361360574999


In [445]:
test_preds = [i if i>0 else 0 for i in list(bst.predict(dtest))]

In [446]:
for_sub = pd.DataFrame({'fullVisitorId' : test['fullVisitorId'],'PredictedLogRevenue' : test_preds})

In [449]:
for_sub_new = for_sub.groupby('fullVisitorId')['PredictedLogRevenue'].sum().reset_index()

In [453]:
for_sub_new['PredictedLogRevenue'] = for_sub_new['PredictedLogRevenue'].apply(lambda x : math.log(x) if x >0 else 0)

In [454]:
for_sub_new.head()

,fullVisitorId,PredictedLogRevenue
0,3038793126460,20.360520
1,9663019041506,18.274337
2,10992980461157,14.168422
3,17196000342279,18.891327
4,18672749561458,20.127484


In [455]:
for_sub_new.shape

(652237, 2)

In [460]:
final_sub = sample_sub[['fullVisitorId']].merge(for_sub_new, on = 'fullVisitorId', how = 'left')

In [461]:
final_sub.head()

,fullVisitorId,PredictedLogRevenue
0,0000000259678714014,19.149890
1,0000049363351866189,21.179907
2,0000053049821714864,19.000419
3,0000059488412965267,16.856821
4,0000085840370633780,18.694290


In [463]:
pd.isnull(final_sub['PredictedLogRevenue']).value_counts()

False    430589
True     186653
Name: PredictedLogRevenue, dtype: int64

In [467]:
final_sub['PredictedLogRevenue'] = [0 if pd.isnull(i) else i for i in final_sub['PredictedLogRevenue'].values]

In [468]:
final_sub.shape

(617242, 2)

In [469]:
pd.isnull(final_sub['PredictedLogRevenue']).value_counts()

False    617242
Name: PredictedLogRevenue, dtype: int64

In [470]:
final_sub.head()

,fullVisitorId,PredictedLogRevenue
0,0000000259678714014,19.149890
1,0000049363351866189,21.179907
2,0000053049821714864,19.000419
3,0000059488412965267,16.856821
4,0000085840370633780,18.694290


In [471]:
final_sub.to_csv('final_sub.csv', index = False)

# Key points

In [ ]:
Each row in the dataset is one visit to the store. 
Predicting the log of the total revenue per user.
Only 11515 transactions in training set has valid transactionRevenue values.
Only 10366 unique users out of 742735 unique users has value for transactionRevenue.
5296 unique users are common between training & test set.
365 unique users are common between train_use (i.e. with non-zero transactions) & test set.
Use regex, nlp on trafficSource_referralPath

# TO THINK

In [ ]:
**If if consider totals_transactionRevenue as zero when it does not exists

# TO-DO

In [ ]:
Matrix to optimize/predict make
To date which is in human readble format 'visitStartTime'
json parse : device, geoNetwork, totals, trafficSource
Time series think as dates are there
Train validation split based on date as it is a sequential data
Target value prepare using log when sum not zero